In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)
print(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

cuda


In [ ]:

input_file = "/content/sentences1.txt"

# Read lines from the input file and store them in a list
with open(input_file, "r") as file:
    sentences_list = [line.strip() for line in file.readlines()]

# Tokenize and encode descriptions
encoded_descriptions = [tokenizer.encode(description, add_special_tokens=True, max_length=512, truncation=True) for description in sentences_list]

In [ ]:
max_len = max(len(desc) for desc in encoded_descriptions)
padded_descriptions = [desc + [0] * (max_len - len(desc)) for desc in encoded_descriptions]

# Convert to PyTorch tensors and move to GPU
input_ids = torch.tensor(padded_descriptions).to(device)


In [ ]:
with torch.no_grad():
    outputs = model(input_ids)
    embeddings = outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token embeddin

similarity_matrix = cosine_similarity(embeddings.cpu().numpy())

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
print("Similarity Matrix:")
print(similarity_matrix)

Similarity Matrix:
[[1.0000005  0.9161375  0.9016785  ... 0.83991563 0.83207595 0.87724197]
 [0.9161375  0.9999999  0.9805208  ... 0.9242056  0.9158429  0.8012941 ]
 [0.9016785  0.9805208  1.         ... 0.9188645  0.90374076 0.8163007 ]
 ...
 [0.83991563 0.9242056  0.9188645  ... 1.         0.9831143  0.7318471 ]
 [0.83207595 0.9158429  0.90374076 ... 0.9831143  0.9999999  0.7239319 ]
 [0.87724197 0.8012941  0.8163007  ... 0.7318471  0.7239319  1.0000001 ]]


In [ ]:
average_similarity = np.mean(similarity_matrix)
print("Average Similarity Score:", average_similarity)

Average Similarity Score: 0.86264783


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util

In [ ]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained SentenceTransformer model onto GPU
model = SentenceTransformer("all-MiniLM-L6-v2").to(device)

# Single list of sentences
input_file = "/content/sentences1.txt"

# Read lines from the input file and store them in a list
with open(input_file, "r") as file:
    sentences_list = [line.strip() for line in file.readlines()]

print(sentences_list)

def contains_invalid_word(sentence):
  invalid_words = ["Properties", "Actions", "Events"]
  for word in invalid_words:
      if word in sentence:
          return True
  return False

# Filter out sentences containing the specified words
filtered_sentences = [sentence for sentence in sentences_list if not (contains_invalid_word(sentence) and len(sentence) > 1)]

# Compute embeddings
embeddings = model.encode(filtered_sentences, convert_to_tensor=True).to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

['SmartLighting Properties:', 'SmartLighting::color: Represents the current color of the light.', 'SmartLighting::brightness: Denotes the intensity of the light emitted.', 'SmartLighting::powerState: Indicates whether the light is turned on or off.', 'SmartLighting Actions:', 'SmartLighting::setColor: Sets the color of the light.', 'SmartLighting::adjustBrightness: Adjusts the brightness level of the light.', 'SmartLighting::turnOnOff: Turns the light on or off.', 'SmartLighting Events:', 'SmartLighting::colorChanged: Triggered when the color of the light is changed.', 'SmartLighting::brightnessChanged: Triggered when the brightness level of the light is adjusted.', 'SmartLighting::powerStateChanged: Triggered when the power state of the light is changed.', '', 'SmartFan Properties:', 'SmartFan::speed: Represents the current speed of the fan (low, medium, high, etc.).', 'SmartFan::rotation: Denotes the rotation speed of the fan blades (if applicable).', 'SmartFan::powerState: Indicates

In [ ]:
filtered_sentences

['SmartLighting::color: Represents the current color of the light.',
 'SmartLighting::brightness: Denotes the intensity of the light emitted.',
 'SmartLighting::powerState: Indicates whether the light is turned on or off.',
 'SmartLighting::setColor: Sets the color of the light.',
 'SmartLighting::adjustBrightness: Adjusts the brightness level of the light.',
 'SmartLighting::turnOnOff: Turns the light on or off.',
 'SmartLighting::colorChanged: Triggered when the color of the light is changed.',
 'SmartLighting::brightnessChanged: Triggered when the brightness level of the light is adjusted.',
 'SmartLighting::powerStateChanged: Triggered when the power state of the light is changed.',
 '',
 'SmartFan::speed: Represents the current speed of the fan (low, medium, high, etc.).',
 'SmartFan::rotation: Denotes the rotation speed of the fan blades (if applicable).',
 'SmartFan::powerState: Indicates whether the fan is turned on or off.',
 'SmartFan::setSpeed: Sets the speed of the fan.',
 

In [ ]:
# cosine_scores = util.cos_sim(embeddings, embeddings)
cosine_scores = torch.matmul(embeddings, embeddings.T).cpu().numpy()
# Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(cosine_scores.shape[0]):
    for j in range(cosine_scores.shape[1]):
        if cosine_scores[i][j] < 0.999:
            pairs.append({"index": [i, j], "score": cosine_scores[i][j]})


In [ ]:
pairs

[{'index': [0, 1], 'score': 0.857345},
 {'index': [0, 2], 'score': 0.76766133},
 {'index': [0, 3], 'score': 0.86677897},
 {'index': [0, 4], 'score': 0.7357327},
 {'index': [0, 5], 'score': 0.7388651},
 {'index': [0, 6], 'score': 0.8654246},
 {'index': [0, 7], 'score': 0.764073},
 {'index': [0, 8], 'score': 0.7684822},
 {'index': [0, 9], 'score': 0.104450345},
 {'index': [0, 10], 'score': 0.44405332},
 {'index': [0, 11], 'score': 0.37192163},
 {'index': [0, 12], 'score': 0.4345861},
 {'index': [0, 13], 'score': 0.36746514},
 {'index': [0, 14], 'score': 0.28485063},
 {'index': [0, 15], 'score': 0.41389155},
 {'index': [0, 16], 'score': 0.38696936},
 {'index': [0, 17], 'score': 0.365637},
 {'index': [0, 18], 'score': 0.40571028},
 {'index': [0, 19], 'score': 0.104450345},
 {'index': [0, 20], 'score': 0.51791143},
 {'index': [0, 21], 'score': 0.532333},
 {'index': [0, 22], 'score': 0.5172593},
 {'index': [0, 23], 'score': 0.4634667},
 {'index': [0, 24], 'score': 0.41477183},
 {'index': [0,

In [ ]:
# pairs = [pair for pair in pairs if not (int(pair["score"]) == 1 or pair["score"] <= 0)]
# pairs = sorted(pairs, key=lambda x: x["score"], reverse=True)

# for pair in pairs[0:10]:
#     i, j = pair["index"]
#     print("{} \t\t {} \t\t Score: {:.4f}".format(
#         sentences_list[i], sentences_list[j], pair["score"]
#     ))

# # Define the file path for output
# output_file = "/content/pairs.txt"

# # Write top pairs to the output file
# with open(output_file, "w") as file:
#     for pair in pairs:
#         i, j = pair["index"]
#         file.write("{}\n{}\nScore: {:.4f}\n\n".format(
#             sentences_list[i], sentences_list[j], pair["score"]
#         ))

# print(f"Top pairs exported to {output_file}")

# Extract device names from sentences
# Define a function to check if a sentence contains any of the specified words

def extract_device_name(sentence):
    return sentence.split("::")[0]

filtered_pairs = []
for pair in pairs:
    index_0, index_1 = pair["index"]
    if index_0 < len(filtered_sentences) and index_1 < len(filtered_sentences):
        device_name_0 = extract_device_name(filtered_sentences[index_0])
        device_name_1 = extract_device_name(filtered_sentences[index_1])
        if device_name_0 != device_name_1:
            filtered_pairs.append(pair)
    else:
        print("Index out of range:", index_0, index_1)

filtered_pairs = sorted(filtered_pairs, key=lambda x: x["score"], reverse=True)
# Print the top 10 filtered pairs
for pair in filtered_pairs[:10]:
    i, j = pair["index"]
    print("{} \t\t {} \t\t Score: {:.4f}".format(
        filtered_sentences[i], filtered_sentences[j], pair["score"]
    ))

# Define the file path for output
output_file = "/content/pairs_new.txt"

# Write top filtered pairs to the output file
with open(output_file, "w") as file:
    for pair in filtered_pairs:
        i, j = pair["index"]
        file.write("{}\n{}\nScore: {:.4f}\n\n".format(
            filtered_sentences[i], filtered_sentences[j], pair["score"]
        ))

print(f"Filtered top pairs exported to {output_file}")



SmartLighting::powerStateChanged: Triggered when the power state of the light is changed. 		 SmartBlinds::powerStateChanged: Triggered when the power state of the blinds is changed. 		 Score: 0.7534
SmartBlinds::powerStateChanged: Triggered when the power state of the blinds is changed. 		 SmartLighting::powerStateChanged: Triggered when the power state of the light is changed. 		 Score: 0.7534
SmartLighting::turnOnOff: Turns the light on or off. 		 SmartFan::turnOnOff: Turns the fan on or off. 		 Score: 0.6993
SmartFan::turnOnOff: Turns the fan on or off. 		 SmartLighting::turnOnOff: Turns the light on or off. 		 Score: 0.6993
SmartLighting::powerState: Indicates whether the light is turned on or off. 		 SmartFan::powerState: Indicates whether the fan is turned on or off. 		 Score: 0.6952
SmartFan::powerState: Indicates whether the fan is turned on or off. 		 SmartLighting::powerState: Indicates whether the light is turned on or off. 		 Score: 0.6952
SmartLighting::powerState: Indicat

In [ ]:
import torch

# Compute the correlation matrix
correlation_matrix = torch.matmul(embeddings, embeddings.T).cpu().numpy()

# Print correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)

Correlation Matrix:
[[1.0000001  0.8053572  0.6880628  ... 0.2812408  0.32055464 0.09974275]
 [0.8053572  0.9999999  0.6830369  ... 0.27025202 0.31648147 0.11530925]
 [0.6880628  0.6830369  1.         ... 0.37200078 0.589153   0.05155164]
 ...
 [0.2812408  0.27025202 0.37200078 ... 1.         0.7470374  0.11044502]
 [0.32055464 0.31648147 0.589153   ... 0.7470374  1.         0.128036  ]
 [0.09974275 0.11530925 0.05155164 ... 0.11044502 0.128036   1.0000001 ]]
